In [1]:
import sys
import os
cwd = os.getcwd().replace('/demo', '')
sys.path.insert(0, cwd)

In [2]:
%%capture
import mercury as mr
import numpy as np
import pandas as pd
from IPython.display import display
import pickle

from lucupy.minimodel.site import ALL_SITES, Site
from lucupy.minimodel.semester import SemesterHalf
from lucupy.observatory.abstract import ObservatoryProperties
from lucupy.observatory.gemini import GeminiProperties

from definitions import ROOT_DIR
from scheduler.core.builder.blueprint import CollectorBlueprint, OptimizerBlueprint
from scheduler.core.builder.builder import SchedulerBuilder
from scheduler.core.components.collector import *
from scheduler.core.output import print_collector_info, plans_table, pickle_plans, pickle_selection
from scheduler.core.programprovider.ocs import read_ocs_zipfile, OcsProgramProvider
from scheduler.core.statscalculator import StatCalculator
from scheduler.services import logger_factory


In [3]:
# set Application parameters
app = mr.App(title="Scheduler Validate Test",
        description="Try the Validation mode for the Scheduler",
        show_code=False,
        show_prompt=True,
        continuous_update=False,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True)

mercury.App

In [4]:
out_dir = mr.OutputDir()
print(f"Output directory path: {out_dir.path}")

mercury.OutputDir

Output directory path: .


In [5]:
def set_dates():

  start_date = mr.Text(value="2018-10-01", label="Start Date", rows=1)
  start_time = mr.Text(value="08:00:00", label="Start Time", rows=1)
  end_date = mr.Text(value="2018-10-03", label="End Date", rows=1)
  end_time = mr.Text(value="08:00:00", label="End Time", rows=1)
  return f'{start_date.value} {start_time.value}', f'{end_date.value} {end_time.value}'

def set_sites():
    selected = mr.MultiSelect(label="Sites", 
                              value=["GS", "GN"], 
                              choices=["GS", "GN"])
    if selected.value:
      if 'GS' in selected.value and 'GN' in selected.value:
         return ALL_SITES
      else:
         return Site(selected.value[0])
    return ALL_SITES # HOW TO HANDLE ERRORS? 


## SCHEDULER 

It does take a while to load everything. When the run button is enable the Scheduler is ready to work. The initial values shown are the default parameters displayed in the sidebar.

In [6]:

ObservatoryProperties.set_properties(GeminiProperties)

# Read in a list of JSON data
programs = read_ocs_zipfile(os.path.join(ROOT_DIR, 'scheduler', 'data', '2018B_program_samples.zip'));

# Create the Collector and load the programs.
collector_blueprint = CollectorBlueprint(
    ['SCIENCE', 'PROGCAL', 'PARTNERCAL'],
    ['Q', 'LP', 'FT', 'DD'],
    1.0
);


start, end = set_dates()
sites = set_sites()


collector = SchedulerBuilder.build_collector(
    start=Time(start, format='iso', scale='utc'),
    end=Time(end, format='iso', scale='utc'),
    sites=sites,
    semesters=frozenset([Semester(2018, SemesterHalf.B)]),
    blueprint=collector_blueprint
);
# Create the Collector and load the programs.
collector.load_programs(program_provider_class=OcsProgramProvider,
                        data=programs)

print_collector_info(collector, samples=10)

selector = SchedulerBuilder.build_selector(collector, hitnum_nights_to_schedule=3);

# Prepare the optimizer.
optimizer_blueprint = OptimizerBlueprint(
    "GreedyMax"
)
optimizer = SchedulerBuilder.build_optimizer(
    blueprint=optimizer_blueprint
)

# The total nights for which visibility calculations have been done.
total_nights = len(collector.time_grid)

# Create the overall plans by night.
overall_plans = {}

for night_idx in range(selector.num_nights_to_schedule):
    # Get the night indices for which we are selecting.
    # TODO: We will want scores for nights to look ahead for greedy optimization.
    # TODO: For now, we use the entire period for which visibility calculations have been done.
    # night_indices = range(night_idx, total_nights)
    night_indices = np.array([night_idx])
    # selection = selector.select(night_indices=np.array([0, 1, 2])
    selection = selector.select(night_indices=night_indices)
    pickle_selection(selection, out_dir.path, night_idx)
    # Run the optimizer to get the plans for the first night in the selection.
    # mr.Md(f'### GreedyMax Output for night {night_idx}')
    plans = optimizer.schedule(selection)
    night_plans = plans[0]

    # Store the plans in the overall_plans array for that night.
    # TODO: This might be an issue. We may need to index nights (plans) in optimizer by night_idx.
    overall_plans[night_idx] = night_plans

    # Perform the time accounting on the plans.
    collector.time_accounting(night_plans)


overall_plans = [p for _, p in sorted(overall_plans.items())]
plan_summary = StatCalculator.calculate_plans_stats(overall_plans,collector)
# print_plans(overall_plans)
night_tables = plans_table(overall_plans)
pickle_plans(night_tables, out_dir.path, start,end)
for n_idx, table_per_site in enumerate(night_tables):
    mr.Md(f"## Night Idx: {n_idx+1} ")
    for site in sites:
        mr.Md(f"### Site: {site.value[0]} ")
        display(table_per_site[site])
mr.Md('## Program Completion')
df_pcompl = pd.DataFrame(plan_summary).T.rename(columns={0:"% Completion",1:"Score"})
display(df_pcompl)

print('DONE')


mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.MultiSelect

...

### Site: Gemini South 

,Start,Observation,Atom start,Atom end,Length,Score,Instrument
0,2018-10-02 20:43:59.999998-03:00,GS-2018B-Q-105-92,0,1,22,90.654853,GMOS-S
1,2018-10-02 21:05:59.999998-03:00,GS-2018B-Q-115-11,0,8,70,2497.122630,GMOS-S
2,2018-10-03 05:16:59.999998-03:00,GS-2018B-Q-120-8,0,3,24,123.964311,GMOS-S
3,2018-10-03 05:40:59.999998-03:00,GS-2018B-Q-223-31,0,62,47,2705.542535,Flamingos2


## Program Completion

,% Completion,Score
GN-2018B-Q-127,100.0%,23564.484695
GN-2018B-Q-104,0.0%,243.526152
GN-2018B-Q-903,100.0%,8311.952498
GN-2018B-Q-230,0.0%,6.186366
GN-2018B-Q-223,14.3%,14.435616
GN-2018B-Q-101,30.8%,1717.416393
GS-2018B-Q-115,55.6%,10387.803293
GS-2018B-Q-206,50.0%,9838.172664
GS-2018B-Q-223,25.0%,5018.470967
GN-2018B-Q-108,0.0%,23.321425


DONE
